# Experiment: _Multi-label splitting_

The node labels for concept maps often (~80% of the time) consist of more than one word. This experiment tests the effect of splitting these labels and then classifying the graphs afterwards.

In [ ]:
from notebook_prelude import *

In [ ]:
NA_VAL = '-'
EXPERIMENT_NAME = 'experiment_split_multi_words'

df = results_helper.get_experiments_by_names([
    EXPERIMENT_NAME + '_with_splitted',
    EXPERIMENT_NAME + '_without_splitter'
])
print('# Results: {}'.format(len(df)))
datasets = df.dataset.unique()
df_text = results_helper.get_results(filter_out_non_complete_datasets=False, only_load_dataset=datasets)
df_text = df_text[(df_text.type == 'text') & (df_text.combined == False)]
df = df.append(df_text).fillna(NA_VAL).reset_index()

In [ ]:
def get_max_grouped_by(df, groupby, attr = 'mean_test_f1_macro'):
    g = df.groupby(groupby)[attr]
    g_ = g.max().to_frame().unstack()
    g__ = g.idxmax()
    return g_, g__

df_ = df[df.type == 'concept_map']
groupby = ['dataset', 'graph_preprocessing']

df_, df__ = get_max_grouped_by(df_, groupby)
df_.columns = df_.columns.droplevel().values
df_ = df_.rename(columns={'-': 'not_split', 'GraphMultiWordLabelSplitter': 'split'})
df_['difference'] = df_['split'] - df_['not_split']

df__ = df__.to_frame().unstack()
df_['split_idx'] = df__.values[:,0]
df_['not_split_idx'] = df__.values[:,1]
df_

for idx, x in enumerate(['split_idx', 'not_split_idx']):
    df___ = df.loc[df_[x]]
    std = df___.std_test_f1_macro
    df_[x + '_std'] = std.values
    
df_
#df.loc[df_.split_idx].mean_test_f1_macro

In [ ]:
df_[['not_split', 'split']]